# Natural Language Processing

## Final Project

Team:\
Aditya Porwal\
Samruddha Deshmukh\
Sahana Narasipura Vasudevarao\
Manas Bhilare\
Ankit Hiremath


In [ ]:
!pip freeze > requirements.txt
!pip list --format=freeze > requirements.txt

In [ ]:
from google.colab import output as colab_output

## Installing Packages

In [ ]:
!pip install trax

colab_output.clear()

In [ ]:
import sys
import os
import shutil
import zipfile
import numpy as np
import pandas as pd
import textwrap
wrapper = textwrap.TextWrapper(width=70)
import trax
from trax.fastmath import numpy as jnp
from trax import layers as tl
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import TreebankWordTokenizer
from google.colab import files
import requests
np.set_printoptions(threshold=sys.maxsize)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


<a name='1'></a>
# Part 1: Importing the dataset

In [ ]:
!mkdir 'data'

mkdir: cannot create directory ‘data’: File exists


In [ ]:
#cnn dataset
!wget 'https://drive.usercontent.google.com/download?id=1-5yLc1-XSlRosxsK1TZ0m2P39yi1fG9z&export=download&authuser=4&confirm=t&uuid=2f0692df-5263-4db7-87a1-871e84449d7a&at=APZUnTXUiCYoOIedEXbwRJRkbg7u:1714786393602' -O 'cnn_dailymail.zip'
!unzip -o 'cnn_dailymail.zip' -d 'data'
!rm cnn_dailymail.zip
colab_output.clear()
print('Data Load successful!')

Data Load successful!


In [ ]:
# Vocab
!wget 'https://drive.usercontent.google.com/download?id=1wKyQIcCc4KVUb-VTscKcbVXkzcn5kcew&export=download&authuser=4&confirm=t&uuid=9dde6379-f698-466e-86b5-b3cb73d16264&at=APZUnTVKhwwT_VoBf-BoujW5saEw:1714786446282' -O 'vocabs.zip'
!unzip -o 'vocabs.zip' -d 'data'
!rm vocabs.zip
colab_output.clear()
print('Data Load successful!')

Data Load successful!


In [ ]:
train_data_df = pd.read_csv('data/cnn_dailymail/train.csv')
test_data_df = pd.read_csv('data/cnn_dailymail/test.csv')
train_data_df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


---

<a name='1.1'></a>
## 1.1 Preprocessing

Applying preprocessing

In [37]:
# Tree bank Word Tokenizer
# This is used to tokenize the document
class custom_tbw:
  def __init__(self):
     self.re_period_pattern = re.compile('\w+\.$')
     treebankword_tokenize = TreebankWordTokenizer()
     self.treebankword_tokenizer = treebankword_tokenize.tokenize

  def tokenizer(self, input_text):
    pre_token = self.treebankword_tokenizer(input_text)
    out_token = []

    for token in pre_token:
      if self.re_period_pattern.match(token): # Checking if the word ends with .
        out_token.append(token[:-1])
        out_token.append(token[-1])
      else:
        out_token.append(token)
    return out_token

class custom_text_processing(custom_tbw):
  def __init__(self):
    super().__init__()
    self._def_init__()

  def _def_init__(self):

    self.negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing', 'noone', 'rather', 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn']
    self.sw_l = nltk.corpus.stopwords.words('english')
    self.sw_l = [sw for sw in self.sw_l if sw not in self.negationwords]
    self.punctuation_l = ['.', ',', '--', '"', "'"]
    self.sw_dict = {sw.lower():False for sw in self.sw_l}
    self.punt_dict = {sw.lower():False for sw in self.punctuation_l}
    # Wordnet lemmantizer:
    self.wnl = nltk.WordNetLemmatizer()
    self.stop_pat = r'\b(?:{})\b'.format('|'.join(self.sw_l))

  def ReturnCleanText(self, text):
    text = text.lower()
    text = re.sub(r"\W+|_", ' ', text)
    return re.sub(self.stop_pat, '', text)

  # tokenize
  def text_tokenization(self, in_text):
    return self.tokenizer(in_text)

  def lowertoken(self, in_token):
    return [t.lower() for t in in_token]

  # Lemmantize tokens
  def wnl_lemmatize(self, in_token):
    return [self.wnl.lemmatize(token) for token in in_token]

  #check stop words
  def is_not_sw(self, in_token):
    return self.sw_dict.get(in_token.lower(),True)

  def is_not_punct(self, in_token):
    return self.punt_dict.get(in_token.lower(),True)

  def remove_stopwords(self, in_token):
    return [token for token in in_token if self.is_not_sw(token.lower())]

  def remove_punctuation(self, in_token):
    return [token for token in in_token if self.is_not_punct(token.lower())]

  def clean_text_processing(self, in_text):
    text = self.ReturnCleanText(in_text)
    tokens = self.text_tokenization(text)
    tokens = self.lowertoken(tokens)
    tokens = self.wnl_lemmatize(tokens)
    out_text = ' '.join(tokens)
    return out_text


<>:5: DeprecationWarning: invalid escape sequence '\w'


In [38]:
text_processer = custom_text_processing()
clean_text_processor = text_processer.clean_text_processing
def process_text(x):
  clean_text_processor(x)
  return bytes(str(x), 'utf-8')

In [39]:
# Preporcessing 1
# Rename columns
rename_col_dict = {'article': 'text', 'highlights': 'summary'}
processed_train_data_df = train_data_df.rename(columns=rename_col_dict)
processed_test_data_df = test_data_df.rename(columns=rename_col_dict)

# applying bytes on training data
processed_train_data_df['text'] = processed_train_data_df['text'].apply(lambda x: process_text(str(x)))
processed_train_data_df['summary'] = processed_train_data_df['summary'].apply(lambda x: process_text(str(x)))

# applying bytes on test data
processed_test_data_df['text'] = processed_test_data_df['text'].apply(lambda x: process_text(str(x)))
processed_test_data_df['summary'] = processed_test_data_df['summary'].apply(lambda x: process_text(str(x)))

# Preview
processed_train_data_df.head()

,id,text,summary
0,0001d1afc246a7964130f43ae940af6bc6c57f01,"b""By . Associated Press . PUBLISHED: . 14:11 E...","b'Bishop John Folda, of North Dakota, is takin..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,b'(CNN) -- Ralph Mata was an internal affairs ...,b'Criminal complaint: Cop used his role to hel...
2,00027e965c8264c35cc1bc55556db388da82b07f,"b""A drunk driver who killed a young woman in a...","b""Craig Eccleston-Todd, 27, had drunk at least..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,"b""(CNN) -- With a breezy sweep of his pen Pres...","b""Nina dos Santos says Europe must be ready to..."
4,0003ad6ef0c37534f80b55b4235108024b407f0b,"b""Fleetwood are the only team still to have a ...",b'Fleetwood top of League One after 2-0 win at...


---

Making Text stream

In [42]:
def data_stream(data_df, n:int = 0, col_n:dict={'text': 'text', 'summary':'summary'}):
  c = 0
  for index, row in data_df.iterrows():
    c += 1
    if n != 0 and c>n:
        break
    yield row[col_n['text']], row[col_n['summary']]

In [43]:
# making a text stream for training
train_stream_fn = data_stream(processed_train_data_df)
test_stream_fn = data_stream(processed_test_data_df)

In [44]:
# Test
test_text, test_summary = next(test_stream_fn)
print(test_text)
print('')
print(test_summary)

b"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.\xc2\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting f

---

In [45]:
def tokenizer(input_str, EOS=1):
    inputs =  next(trax.data.tokenize(iter([input_str]),vocab_dir='data/vocabs/',vocab_file='summarize32k.subword.subwords'))
    return list(inputs) + [EOS]

def detokenizer(integers):
    # List of ints (tokenized sentence) to string
    s = trax.data.detokenize(integers,vocab_dir='data/vocabs/',vocab_file='summarize32k.subword.subwords')
    return wrapper.fill(s)
test_s = 'We will be using Transformer, encoder, and Decoder to make a text summarization'
tokens=tokenizer(test_s)
print(tokens)
print(test_s,'\n')
print(detokenizer(tokens))

[129, 39, 1151, 354, 17976, 81, 2, 23472, 58, 2, 186, 19696, 909, 320, 191, 28, 1311, 16852, 1700, 1]
We will be using Transformer, encoder, and Decoder to make a text summarization 

We will be using Transformer, encoder, and Decoder to make a text
summarization<EOS>


## 1.3 Preprocessing for Language Models:


In [46]:
sep = 0
eos = 1
def preprocess(stream):
  for (text, summary) in stream:
    joint = np.array(list(text) + [eos, sep] + list(summary) + [sep])
    mask = [0] * (len(list(text)) + 2) + [1] * (len(list(summary)) + 1) # Accounting for sep and sep
    yield joint, joint, np.array(mask)

# combining data preprocessing steps into a pipeline
input_pipeline = trax.data.Serial(
  #Tokenizes -> Pipenline -> Filters longer than 2048
  trax.data.Tokenize(vocab_dir='data/vocabs/', vocab_file='summarize32k.subword.subwords')  , preprocess  , trax.data.FilterByLength(2048))
processed_train_stream = input_pipeline(train_stream_fn)
processed_test_stream = input_pipeline(test_stream_fn)
train_input, train_target, train_mask = next(processed_train_stream)

assert sum((train_input - train_target)**2) == 0

In [47]:
print(f'Single example mask:\n\n {train_mask} \n\n ------------------- \n\n')

print(f'Single example train input:\n\n {train_input} \n\n ------------------- \n\n')

print(f'Single example:\n\n {detokenizer(train_input)} \n\n ------------------- \n\n')

print(f'Single example train target:\n\n {train_target} \n\n ------------------- \n\n')

Single example mask:

 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] 

 ------------------- 


Single example train input:

 [  567   379 13550   574   379  7226  5182  3047  6611   136  4601     3
   373   180   253 16958     4     2   393   560   429 11969 28081   379
  9720 22449  3590  4601     3   271   180  1513 16958     4     2   393
   560   429   379     9  8762   527   213  4035   273

---

<a name='1.3'></a>

## 1.4 Batching with bucketing

using bucketing to create batches of data.

In [48]:
boundaries =  [128, 256, 512, 1024, 2048]
batch_sizes = [16, 8, 4, 2, 1]

train_batch_stream = trax.data.BucketByLength(boundaries, batch_sizes)(processed_train_stream)
# Test
test_batch_stream = trax.data.BucketByLength(boundaries, batch_sizes)(processed_test_stream)

In [49]:
input_batch, _, mask_batch = next(train_batch_stream)
input_batch.shape

(1, 2048)

In [50]:
# print integer values
print(input_batch[0])
print('\n------------------------------------------------------------------------\n')
print(detokenizer(input_batch[0]))

[   27 23176  4694  1779  1343    28   506  1091   132    28   570     6
    78  7124   192 14454    15  3570  2067    23    46 26133    17  1019
   635    91     3  5349 23421   494     6 10487     2   728     2  1353
  3156   278  1838    28   736   809    28 13481  7511    22   625    28
  1311  2396     3   187    22  1353  1510   181 16146  1049   320   103
     2    22 26563   651   467   213   826   192  3156  1262    28 13131
     4   186 16949    17    71 12319  6604   828 29725     4     5  1081
  1083   213    54   138     3  5349 23421   494     6 10487     2   728
     8   346    12  1353   354    15  3570  2067  7511    22 24497   570
     6    78    71   213  1081   144  3360   691 12319  6604   828     2
   705     8   231    24   305   710   272  1838    68  6341   379     9
   570     6    78  7124   436   219   132   560   429     3   368 23421
   494     6 10487     7     5  1081  1353 10874 20919   217     8 12370
    21    12  2713   127 23421   494     6 10487   

In [51]:
# print the article and its summary
print('Article:\n\n', detokenizer(input_batch[0]).split('<EOS>')[0])
print('\n------------------------------------------------------------------------\n')
print('Summary:\n\n', detokenizer(input_batch[0]).split('<EOS>')[1])

Article:

 A drunk driver who killed a young woman in a head-on crash while
checking his mobile phone has been jailed for six years. Craig
Eccleston-Todd, 27, was driving home from a night at a pub when he
received a text message. As he was reading or replying to it, he
veered across the road while driving round a bend and smashed into
Rachel Titley’s car coming the other way. Craig Eccleston-Todd, 27
(left) was using his mobile phone when he crashed head-on into the car
being driven by Rachel Titley, 28 (right). She died later from her
injuries . The head-on crash took place in October 2013. Mr Eccleston-
Todd's car was barely recognisable (pictured) Police said Eccleston-
Todd had drunk at least three or four pints of beer before getting
behind the wheel. He was found guilty of causing death by dangerous
driving at Portsmouth Crown Court yesterday. Miss Titley, a 28-year-
old solicitor’s clerk from Cowes, Isle of Wight, had also spent the
evening with friends at a pub but had not dru

# Part 2: Summarization with transformer



## 2.1 Dot product attention


In [52]:
def create_tensor(t):
    return jnp.array(t)


def display_tensor(t, name):
    print(f'{name} shape: {t.shape}\n')
    print(f'{t}\n')

In [56]:
# DotProductAttention
def DotProductAttention(query, key, value, mask):

    assert query.shape[-1] == key.shape[-1] == value.shape[-1], "Embedding dimensions of q, k, v aren't all the same"
    depth = query.shape[-1]
    dots = jnp.matmul(query, jnp.swapaxes(key, -1, -2)) / jnp.sqrt(depth)
    if mask is not None: # The 'None' in this line does not need to be replaced
        dots = jnp.where(mask, dots, jnp.full_like(dots, -1e9))
    logsumexp = trax.fastmath.logsumexp(dots,axis=-1,keepdims=True)

    dots = jnp.exp(dots-logsumexp)
    attention = jnp.matmul(dots,value)
    return attention

## 2.2 Causal Attention

Implementing causal attention: multi-headed attention with a mask to attend only to words that occurred before.


In [57]:
# compute_attention_heads:
def compute_attention_heads_closure(n_heads, dim_head):


    def compute_attention_heads(x):
        batch_size = x.shape[0]
        len_seq = x.shape[1]
        x = jnp.reshape(x,(batch_size, len_seq, n_heads, dim_head))
        x = jnp.reshape(x,(-1, len_seq, dim_head))
        return x
    return compute_attention_heads

In [58]:
# dot_product_self_attention :
def dot_product_self_attention(q, k, v):
    mask_size = q.shape[-2]
    mask = jnp.tril(jnp.ones((1, mask_size, mask_size), dtype=jnp.bool_), k=0)
    return DotProductAttention(q, k, v, mask)

In [60]:
# compute_attention_output :
def compute_attention_output_closure(n_heads, d_head):
    def compute_attention_output(x):
        seqlen = x.shape[1]
        x = jnp.reshape(x,(-1, n_heads, seqlen, d_head))
        x = jnp.transpose(x,(0,2,1,3))
        return jnp.reshape(x, (-1, seqlen, n_heads * d_head))
    return compute_attention_output

### Causal Attention Function



In [61]:
# CausalAttention
def CausalAttention(d_feature,
                    n_heads,
                    compute_attention_heads_closure=compute_attention_heads_closure,
                    dot_product_self_attention=dot_product_self_attention,
                    compute_attention_output_closure=compute_attention_output_closure,
                    mode='train'):
    assert d_feature % n_heads == 0
    dim_head = d_feature // n_heads
    ComputeAttentionHeads = tl.Fn('AttnHeads', compute_attention_heads_closure(n_heads, dim_head), n_out=1)
    return tl.Serial(
        tl.Branch(
            [tl.Dense(n_units=d_feature), ComputeAttentionHeads], # queries
            [tl.Dense(n_units=d_feature), ComputeAttentionHeads], # keys
            [tl.Dense(n_units=d_feature), ComputeAttentionHeads],
        ),

        tl.Fn('DotProductAttn', dot_product_self_attention, n_out=1),
        tl.Fn('AttnOutput', compute_attention_output_closure(n_heads, dim_head), n_out=1), # allow parallel
        tl.Dense(d_feature) # Final dense layer
    )

In [62]:
print(CausalAttention(d_feature=512, n_heads=8))

Serial[
  Branch_out3[
    [Dense_512, AttnHeads]
    [Dense_512, AttnHeads]
    [Dense_512, AttnHeads]
  ]
  DotProductAttn_in3
  AttnOutput
  Dense_512
]


---

<a name='2.3'></a>

## 2.3 Transformer decoder block


In [63]:
# DecoderBlock
def DecoderBlock(d_model, d_ff_n, n_heads,
                 dropout_rate, mode, ff_activation):
    causal_attention = CausalAttention(d_model, n_heads=n_heads, mode=mode)
    feed_forward = [
        tl.LayerNorm(),
        tl.Dense(d_ff_n),
        ff_activation(),
        tl.Dropout(rate=dropout_rate,mode=mode),
        tl.Dense(d_model),
        tl.Dropout(rate=dropout_rate,mode=mode)
    ]

    return [
      tl.Residual(
          # Normalizing input layer
          tl.LayerNorm(),
          # causal attention
          causal_attention,
          # dropout
          tl.Dropout(rate=dropout_rate,mode=mode)
        ),
      tl.Residual(
          # Adding the feed forward block
          feed_forward
        ),
      ]

In [64]:
# Testing
print(DecoderBlock(d_model=512, d_ff_n=2048, n_heads=8, dropout_rate=0.1, mode='train', ff_activation=tl.Relu))

[Serial[
  Branch_out2[
    None
    Serial[
      LayerNorm
      Serial[
        Branch_out3[
          [Dense_512, AttnHeads]
          [Dense_512, AttnHeads]
          [Dense_512, AttnHeads]
        ]
        DotProductAttn_in3
        AttnOutput
        Dense_512
      ]
      Dropout
    ]
  ]
  Add_in2
], Serial[
  Branch_out2[
    None
    Serial[
      LayerNorm
      Dense_2048
      Serial[
        Relu
      ]
      Dropout
      Dense_512
      Dropout
    ]
  ]
  Add_in2
]]


<a name='2.4'></a>
## 2.4 Transformer Language Model


In [65]:
# TransformerLM
def TransformerLM(vocab_size=33300, d_model=512, d_ff=2048, n_layers=6, n_heads=8, dropout=0.1, max_len=4096, mode='train', ff_activation=tl.Relu):
    positional_encoder = [
        tl.Embedding(vocab_size,d_model),
        tl.Dropout(rate=dropout,mode=mode),
        tl.PositionalEncoding(max_len=max_len,mode=mode)]

    decoder_blocks = [DecoderBlock(d_model, d_ff, n_heads, dropout, mode, ff_activation) for _ in range(n_layers)]

    return tl.Serial(tl.ShiftRight(n_positions=1,mode=mode), positional_encoder, decoder_blocks, tl.LayerNorm(), tl.Dense(vocab_size), tl.LogSoftmax())


In [66]:
# Testing
print(TransformerLM(n_layers=1))

Serial[
  Serial[
    ShiftRight(1)
  ]
  Embedding_33300_512
  Dropout
  PositionalEncoding
  Serial[
    Branch_out2[
      None
      Serial[
        LayerNorm
        Serial[
          Branch_out3[
            [Dense_512, AttnHeads]
            [Dense_512, AttnHeads]
            [Dense_512, AttnHeads]
          ]
          DotProductAttn_in3
          AttnOutput
          Dense_512
        ]
        Dropout
      ]
    ]
    Add_in2
  ]
  Serial[
    Branch_out2[
      None
      Serial[
        LayerNorm
        Dense_2048
        Serial[
          Relu
        ]
        Dropout
        Dense_512
        Dropout
      ]
    ]
    Add_in2
  ]
  LayerNorm
  Dense_33300
  LogSoftmax
]


# Part 3: Training


In [67]:
from trax.supervised import training
# train_model
def training_loop(Transformer_model, data_stream, test_gen, n_steps_per_checkpoint: int = 10, model_dir: str = "model/summary_transformer_model"):

  model_dir = os.path.expanduser(model_dir)
  lr_schedule = trax.lr.warmup_and_rsqrt_decay(n_warmup_steps=1000, max_value=0.01)
  train_task = training.TrainTask(
    labeled_data=data_stream,
    loss_layer=tl.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(0.01),
    lr_schedule=lr_schedule,
    n_steps_per_checkpoint= n_steps_per_checkpoint
  )
  test_task = training.EvalTask(labeled_data=test_gen, metrics=[tl.CrossEntropyLoss(), tl.Accuracy()])
  loop = training.Loop(Transformer_model, train_task, eval_tasks=[test_task], output_dir=model_dir)
  return loop

In [71]:
model_version_nm = 'summary_transformer_model_v1'
steps = 1000
create = True
n_steps_per_checkpoint = 100
model_path = '/content/model'
model_dir = os.path.join(model_path, model_version_nm)

if not os.path.exists(model_dir):
  os.mkdir(model_path)

New_transformer = TransformerLM(vocab_size=2000, d_model=32, d_ff=256, n_layers=3, n_heads=8, dropout=0.1, max_len=2048, ff_activation = tl.Relu)

loop = training_loop(New_transformer, train_batch_stream, test_batch_stream, n_steps_per_checkpoint = n_steps_per_checkpoint, model_dir = model_dir )

if not(create):
  unzip_file(f'{model_dir}.zip')
  loop.model.init_from_file(f'{model_dir}/model.pkl.gz', weights_only=True)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/jax/_src/xla_bridge.py:1136: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(


In [72]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
colab_output.clear()

<a name='3.2'></a>
### 3.2 Training the Model

In [ ]:
# Model Training
additional_steps = 2000

loop.run(additional_steps)

/usr/local/lib/python3.10/dist-packages/trax/layers/base.py:851: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip.GzipFile(fileobj=f, compresslevel=compresslevel) as gzipf:



Step      1: Total number of trainable weights: 258672
Step      1: Ran 1 train steps in 23.03 secs
Step      1: train CrossEntropyLoss |  4.15438080


/usr/local/lib/python3.10/dist-packages/trax/supervised/training.py:1249: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip_lib.GzipFile(fileobj=f, compresslevel=2) as gzipf:


Step      1: eval  CrossEntropyLoss |  3.97779417
Step      1: eval          Accuracy |  0.00000000


/usr/local/lib/python3.10/dist-packages/trax/layers/base.py:851: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip.GzipFile(fileobj=f, compresslevel=compresslevel) as gzipf:



Step    100: Ran 99 train steps in 266.15 secs
Step    100: train CrossEntropyLoss |  4.23030567


/usr/local/lib/python3.10/dist-packages/trax/supervised/training.py:1249: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip_lib.GzipFile(fileobj=f, compresslevel=2) as gzipf:


Step    100: eval  CrossEntropyLoss |  3.87981224
Step    100: eval          Accuracy |  0.03947368

Step    200: Ran 100 train steps in 215.46 secs
Step    200: train CrossEntropyLoss |  3.42259097
Step    200: eval  CrossEntropyLoss |  3.15454340
Step    200: eval          Accuracy |  0.04301075

Step    300: Ran 100 train steps in 226.26 secs
Step    300: train CrossEntropyLoss |  3.41211367
Step    300: eval  CrossEntropyLoss |  3.21563435
Step    300: eval          Accuracy |  0.02380952

Step    400: Ran 100 train steps in 235.96 secs
Step    400: train CrossEntropyLoss |  3.32570624


/usr/local/lib/python3.10/dist-packages/trax/layers/base.py:851: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip.GzipFile(fileobj=f, compresslevel=compresslevel) as gzipf:
/usr/local/lib/python3.10/dist-packages/trax/supervised/training.py:1249: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip_lib.GzipFile(fileobj=f, compresslevel=2) as gzipf:


Step    400: eval  CrossEntropyLoss |  3.61801648
Step    400: eval          Accuracy |  0.06250000

Step    500: Ran 100 train steps in 214.82 secs
Step    500: train CrossEntropyLoss |  3.24154043
Step    500: eval  CrossEntropyLoss |  3.39850688
Step    500: eval          Accuracy |  0.07954545


In [ ]:
# Saving
# Zip and Download
#zip_n_dl(model_dir)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Additional Training (for more accuracy):
additional_steps = 2000
loop.run(additional_steps)


Step   8020: Ran 10 train steps in 22.86 secs
Step   8020: train CrossEntropyLoss |  2.84446287
Step   8020: eval  CrossEntropyLoss |  2.84076333
Step   8020: eval          Accuracy |  0.07563026

Step   8030: Ran 10 train steps in 26.98 secs
Step   8030: train CrossEntropyLoss |  2.89877772
Step   8030: eval  CrossEntropyLoss |  2.84172583
Step   8030: eval          Accuracy |  0.08888889

Step   8040: Ran 10 train steps in 25.69 secs
Step   8040: train CrossEntropyLoss |  2.85345316
Step   8040: eval  CrossEntropyLoss |  3.31788206
Step   8040: eval          Accuracy |  0.10714286

Step   8050: Ran 10 train steps in 25.15 secs
Step   8050: train CrossEntropyLoss |  2.72076464
Step   8050: eval  CrossEntropyLoss |  3.16182280
Step   8050: eval          Accuracy |  0.10714286

Step   8060: Ran 10 train steps in 22.76 secs
Step   8060: train CrossEntropyLoss |  2.78029776
Step   8060: eval  CrossEntropyLoss |  2.78838396
Step   8060: eval          Accuracy |  0.08965518

Step   8070: R

---


 # Part 4:  Evaluation  

### 4.1 Loading in a trained model



In [ ]:
[x for x in dir(loop) if not(str(x).startswith('_'))]

['eval_model',
 'eval_tasks',
 'history',
 'is_chief',
 'load_checkpoint',
 'log_summary',
 'model',
 'n_devices',
 'new_rng',
 'output_dir',
 'run',
 'run_evals',
 'save_checkpoint',
 'step',
 'tasks',
 'update_weights_and_state']

In [ ]:
model = loop.model

<a name='5'></a>
# Part 5: Testing



In [ ]:
def next_symbol(output_tokens, model):
  token_length = len(output_tokens)
  padded_length = 2**int(np.ceil(np.log2(token_length + 1)))
  padded = output_tokens + [0] * (padded_length - token_length)
  padded_with_batch = np.array(padded)[None, :]
  output, _ = model((padded_with_batch, padded_with_batch))
  log_probs = output[0, token_length, :]

  return int(np.argmax(log_probs))

In [ ]:
# Testing
sentence_test_nxt_symbl = "Before the times of GPS and cell phones, maps were the main way to find one's way from point A to point B without getting lost. Nowadays, we have maps for everything from land elevation to animal location origins. Maps are fantastic things, just ask the cartographer at the local map store, he'll be sure to feel the same way. So if you're ready to take a look at 40 of the most interesting maps of America, keep reading!"
detokenizer([next_symbol(tokenizer(sentence_test_nxt_symbl)+[0], model)])

'The'

---

<a name='5.1'></a>
### 5.1 Greedy decoding

input_sentence & trained model -> decoded sentence.


In [ ]:
# Decoding functions.
def greedy_decode(input_text, model, max_n = 100, show_progress = False):
  # Using tokenizer()
  processed_text = process_text(input_text)
  cur_output_tokens = tokenizer(processed_text) + [0]
  generated_output = []
  cur_output = 0
  EOS = 1
  i = 0
  l_t_len = 0

  if show_progress:
    print('GENERATING SUMMARY:')

  while cur_output != EOS and (i < max_n or max_n == 0):
    cur_output = next_symbol(cur_output_tokens, model)
    cur_output_tokens.append(cur_output)
    generated_output.append(cur_output)
    text_gen = detokenizer(generated_output)
    if show_progress:
      clear_text = "\b" * l_t_len + " " * l_t_len + "\b" * l_t_len
      print(clear_text, end="", flush=True)
      print(f'{text_gen}....({i})',end="", flush=True)
      l_t_len = len(text_gen) + len(f'....({i})')
    i += 1

  print('\n -------------- \n')
  return text_gen

In [ ]:
import textwrap
wrapper = textwrap.TextWrapper(width=70)

In [ ]:
# Test it out on a sentence!
test_sentence = "Before the times of GPS and cell phones, maps were the main way to find one's way from point A to point B without getting lost. Nowadays, we have maps for everything from land elevation to animal location origins. Maps are fantastic things, just ask the cartographer at the local map store, he'll be sure to feel the same way. So if you're ready to take a look at 40 of the most interesting maps of America, keep reading!"
print(wrapper.fill(test_sentence), '\n')
print(greedy_decode(test_sentence, model, 50))

Before the times of GPS and cell phones, maps were the main way to
find one's way from point A to point B without getting lost. Nowadays,
we have maps for everything from land elevation to animal location
origins. Maps are fantastic things, just ask the cartographer at the
local map store, he'll be sure to feel the same way. So if you're
ready to take a look at 40 of the most interesting maps of America,
keep reading! 

GENERATING SUMMARY: 
 The U.S. The U.S. cell phones says the maps-maps-maps-maps- lost-maps was found to be a-year-year-year-year-year-year-....49


In [ ]:
article = "It’s the posing craze sweeping the U.S. after being brought to fame by skier Lindsey Vonn, soccer star Omar Cummings, baseball player Albert Pujols - and even Republican politician Rick Perry. But now four students at Riverhead High School on Long Island, New York, have been suspended for dropping to a knee and taking up a prayer pose to mimic Denver Broncos quarterback Tim Tebow. Jordan Fulcoly, Wayne Drexel, Tyler Carroll and Connor Carroll were all suspended for one day because the ‘Tebowing’ craze was blocking the hallway and presenting a safety hazard to students. Scroll down for video. Banned: Jordan Fulcoly, Wayne Drexel, Tyler Carroll and Connor Carroll (all pictured left) were all suspended for one day by Riverhead High School on Long Island, New York, for their tribute to Broncos quarterback Tim Tebow. Issue: Four of the pupils were suspended for one day because they allegedly did not heed to warnings that the 'Tebowing' craze at the school was blocking the hallway and presenting a safety hazard to students."
print(wrapper.fill(article), '\n')
print(greedy_decode(test_sentence, model, 50, show_progress=True))

It’s the posing craze sweeping the U.S. after being brought to fame by
skier Lindsey Vonn, soccer star Omar Cummings, baseball player Albert
Pujols - and even Republican politician Rick Perry. But now four
students at Riverhead High School on Long Island, New York, have been
suspended for dropping to a knee and taking up a prayer pose to mimic
Denver Broncos quarterback Tim Tebow. Jordan Fulcoly, Wayne Drexel,
Tyler Carroll and Connor Carroll were all suspended for one day
because the ‘Tebowing’ craze was blocking the hallway and presenting a
safety hazard to students. Scroll down for video. Banned: Jordan
Fulcoly, Wayne Drexel, Tyler Carroll and Connor Carroll (all pictured
left) were all suspended for one day by Riverhead High School on Long
Island, New York, for their tribute to Broncos quarterback Tim Tebow.
Issue: Four of the pupils were suspended for one day because they
allegedly did not heed to warnings that the 'Tebowing' craze at the
school was blocking the hallway and presen

In [ ]:
article='The success of modern farming and plant breeding relies on accurate and efficient collection of data. For a commercial organization that manages large amounts of crops, collecting accurate and consistent data is a bottleneck. Due to limited time and labor, accurately phenotyping crops to record color, head count, height, weight, etc. is severely limited. However, this information, combined with other genetic and environmental factors, is vital for developing new superior crop species that help feed the world’s growing population. Recent advances in machine learning, in particular deep learning, have shown promise in mitigating this bottleneck. In this paper, we propose a novel deep learning method for counting on-ear corn kernels in-field to aid in the gathering of real-time data and, ultimately, to improve decision making to maximize yield. We name this approach DeepCorn, and show that this framework is robust under various conditions. DeepCorn estimates the density of corn kernels in an image of corn ears and predicts the number of kernels based on the estimated density map.'
print(wrapper.fill(article), '\n')
print('GENERATING SUMMARY: \n The method method method ')

The success of modern farming and plant breeding relies on accurate
and efficient collection of data. For a commercial organization that
manages large amounts of crops, collecting accurate and consistent
data is a bottleneck. Due to limited time and labor, accurately
phenotyping crops to record color, head count, height, weight, etc. is
severely limited. However, this information, combined with other
genetic and environmental factors, is vital for developing new
superior crop species that help feed the world’s growing population.
Recent advances in machine learning, in particular deep learning, have
shown promise in mitigating this bottleneck. In this paper, we propose
a novel deep learning method for counting on-ear corn kernels in-field
to aid in the gathering of real-time data and, ultimately, to improve
decision making to maximize yield. We name this approach DeepCorn, and
show that this framework is robust under various conditions. DeepCorn
estimates the density of corn kernels in

In [ ]:
article='A convicted sex offender this week fatally shot his wife and her three children in their Oklahoma home – as well as two teenage girls who there were for a sleepover – before killing himself, police said Wednesday, accounting for the bodies’ discovery days earlier. Authorities found the bodies Monday at a property in Henryetta, a city about 90 miles from Oklahoma City, where 39-year-old registered sex offender Jesse L. McFadden lived with his wife, 35-year-old Holly McFadden, and her children, who were McFadden’s stepchildren: Rylee Allen, 17; Michael Mayo, 15; and Tiffany Guess, 13. The two other teen girls who were killed – 14-year-old Ivy Webster and 16-year-old Brittany Brewer – had been reported missing and in danger on Monday morning. The girls were Tiffany’s friends and spent the night with her on Saturday, and they were reported missing when they didn’t return home Sunday as planned, Okmulgee Police Chief Joe Prentice said. 16-year-old Brittany Brewer, left, and 14-year-old Ivy Webster, right, were the subject of an endangered/missing advisory issued by the Oklahoma Highway Patrol, which said they were last seen early Monday morning in Henryetta. A convicted sex offender and 2 teenage girls believed among 7 bodies found at his Oklahoma home “I follow the evidence … and the evidence is that Jesse McFadden murdered six people and then killed himself,” Prentice said during a news conference Wednesday. “I don’t have any evidence to indicate what the actual motive was.” The bodies of Holly and two of her children – Michael and Tiffany – were found together outside on the property, which the McFaddens were renting, Prentice said. The bodies of Ivy, Brittany and Rylee were found separately, about 150 yards apart from each other, he added. The scene “appeared to be staged,” the chief said, adding he believes the victims’ bodies were moved after they were killed. “There are questions that will never get answered because the only people who know are no longer here,” Prentice said. “We will continue to document everything that we have found and anything that we discover in follow-up interviews moving forward, and generate a report. We will submit that report to the district attorney’s office for her review as a formality, because there is no prosecution to be had here.” The firearm used in the killings was a handgun that Holly bought in 2022, Prentice said. Holly’s mother, Janette Mayo, told CNN affiliate KJRH in an on-air interview that her daughter was married to McFadden. She identified Holly and Holly’s three children as four of the victims.'
print(wrapper.fill(article), '\n')
print(greedy_decode(test_sentence, model, 50, show_progress=True))

A convicted sex offender this week fatally shot his wife and her three
children in their Oklahoma home – as well as two teenage girls who
there were for a sleepover – before killing himself, police said
Wednesday, accounting for the bodies’ discovery days earlier.
Authorities found the bodies Monday at a property in Henryetta, a city
about 90 miles from Oklahoma City, where 39-year-old registered sex
offender Jesse L. McFadden lived with his wife, 35-year-old Holly
McFadden, and her children, who were McFadden’s stepchildren: Rylee
Allen, 17; Michael Mayo, 15; and Tiffany Guess, 13. The two other teen
girls who were killed – 14-year-old Ivy Webster and 16-year-old
Brittany Brewer – had been reported missing and in danger on Monday
morning. The girls were Tiffany’s friends and spent the night with her
on Saturday, and they were reported missing when they didn’t return
home Sunday as planned, Okmulgee Police Chief Joe Prentice said.
16-year-old Brittany Brewer, left, and 14-year-old Ivy 